<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/BERTGCNAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setup and Data Loading



In [1]:
# Install dependencies
!pip install torch torch-geometric transformers imbalanced-learn -q

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy import io as sio

# Mount Google Drive
drive.mount('/content/drive')

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')

# Extract network features
network_features = df[['share_count', 'reaction_count', 'comment_count']].values  # (2282, 3)

# Standardize network features
scaler = StandardScaler()
X_net_std = scaler.fit_transform(network_features)  # (2282, 3)

# Save standardized features
sio.savemat('network.mat', {'X_net_std': X_net_std})
print("Network features shape:", X_net_std.shape)

# Prepare labels (binary classification)
labels = df['Rating'].apply(lambda x: 0 if x == 'mostly true' else 1).values  # 0: mostly true, 1: others
y = np.array(labels)
print("Label distribution:", np.bincount(y))  # [1669, 613]

# Move files to Google Drive
!mv /content/network.mat /content/drive/MyDrive/Projects/Hayat/
print("Network features saved to Google Drive")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.8 MB/s eta 0:00:00
Mounted at /content/drive
Network features sha

# Step 2: Social Branch (Graph Construction and GCN Embeddings)



In [2]:
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops

# Load standardized network features
X_net_std = sio.loadmat('/content/drive/MyDrive/Projects/Hayat/network.mat')['X_net_std']

# Construct graph
G = nx.Graph()
for idx in range(len(df)):
    G.add_node(idx, features=X_net_std[idx])
account_groups = df.groupby('account_id').indices
for account_id, indices in account_groups.items():
    indices = list(indices)
    for i in range(len(indices)):
        for j in range(i + 1, len(indices)):
            G.add_edge(indices[i], indices[j])
print("Graph nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())

# Prepare GNN data
edges = list(G.edges)
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
x = torch.tensor(X_net_std, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
data.edge_index = edge_index
print("GNN Data:", data)

# Define GCN model
class GCN(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64, out_channels=128):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Compute GCN embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gcn_model = GCN().to(device)
data = data.to(device)
gcn_model.eval()
with torch.no_grad():
    gcn_embeddings = gcn_model(data)  # (2282, 128)
print("GCN Embeddings shape:", gcn_embeddings.shape)

# Save GCN model and embeddings
torch.save(gcn_model.state_dict(), 'gcn_model.pth')
torch.save(gcn_embeddings.cpu(), 'gcn_embeddings.pt')
print("GCN model and embeddings saved")

# Move to Google Drive
!mv /content/gcn_model.pth /content/drive/MyDrive/Projects/Hayat/
!mv /content/gcn_embeddings.pt /content/drive/MyDrive/Projects/Hayat/
print("GCN model and embeddings moved to Google Drive")

Graph nodes: 2282 Edges: 368312
GNN Data: Data(x=[2282, 3], edge_index=[2, 370594])
GCN Embeddings shape: torch.Size([2282, 128])
GCN model and embeddings saved
GCN model and embeddings moved to Google Drive
